# Swiss Medical Challenge 2022

Given a dataset of 500,000 records containing contact information (email 
addresses and telephone numbers) of individual clients and companies, 
the job is to build a contactability index (trust index) for each contact 
record. 

The data is provided as is, taken from several company data sources: this means 
that data entries are with many typing errors and mistakes.

_**Important note**: All private information has been removed from both the
code and documents published._

The original dataset contains the following columns, meaning:

* 'ID' = Client ID (several individuals can belong to one client, this is the 
case of company clients)
* 'ID_Mail' = Unique email identifier
* 'Email' = Email address for an individual
* 'Origen_mail' = Source from where that email comes from
* 'ID_Telefono' = Unique phone number identifier
* 'Numero_Completo' = Telephone number for an individual
* 'Origen_telefono' = Source from where that telephone number comes from

## Solution

The task at hand was divided into the following steps:

1. Evaluating the `domain` part of the email address, and assigning a trust 
index. This is done in Batch 1 and first part of Batch 2.
2. Building a trust index for the `user` part of the email address. Second 
part of Batch 2.
3. Cleaning telephone numbers and building a trust index for them. Batch 3.
4. Batch 4 takes care of joining the results in a new, ordered and clean 
dataset that may be used as a contact database having the following 
contactability indexes:

* For emails, the contactability index is built as `domain_trust @ user_trust`.
* For phones, the contactability index is a `trust_number`.

5. In Batch 5 the collected information is put into plot graphs that are 
later used to build the presentation.


#### A few notes:

1. All markdown titles were kept as comments within each code cell.
2. At the beginning and at the end of each cell a copy of the dataframe being 
modified is made in order to allow for several executions of that cell without 
running the previous part of the ipynb. 
3. Along the code some data is being stored separately in order to be used 
in Batch 5 for the plot graphs.

In [89]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt 
plt.style.use('seaborn-colorblind')
%matplotlib inline
import seaborn as sns
import regex as re
from Levenshtein import distance as lev
from concurrent.futures import ThreadPoolExecutor
from dns import resolver
import requests
import threading
from datetime import datetime
import pickle

# Omitir warnings
import warnings
warnings.filterwarnings('ignore')

In [90]:
str(datetime.now())

'2022-09-11 14:23:09.477764'

In [91]:
# Data read

df_data = pd.read_csv('set_smg.csv', encoding='ansi', dtype='object')

df_data.columns

Index(['ID', 'ID_Mail', 'Email', 'Origen_mail', 'ID_Telefono',
       'Numero_Completo', 'Origen_telefono'],
      dtype='object')

In [ ]:
# Data sort

df_data.sort_values(by=['ID', 'ID_Mail'], inplace=True)

# df_data

In [ ]:
# Mails extraction, domains extraction

df_mails = df_data[['ID', 'ID_Mail', 'Origen_mail', 'Email']]

pattern = re.compile(r'(?i)(?P<user>.+)@(?P<domain>.+)')

matches = df_mails.Email.apply(lambda x: pattern.search(x.lower()))
mail_user = matches.apply(lambda x: np.nan if x is None else \
                            x.group('user').lower())
mail_domain = matches.apply(lambda x: np.nan if x is None else \
                            x.group('domain').lower())

df_mails['user'] = mail_user
df_mails['domain'] = mail_domain

df_temp_mails = df_mails.copy()

# df_mails

In [ ]:
df_mails = df_temp_mails.copy()

# Mail duplicates removal

df_mails.drop_duplicates(keep='first', inplace=True)
df_mails.reset_index(drop=True, inplace=True)

df_temp_mails = df_mails.copy()

# df_mails

In [95]:
df_mails = df_temp_mails.copy()

# Domains unique extraction and df setting up

domains = df_mails.domain.unique()
domains_list = list(domains)

df_domains = pd.DataFrame(index=domains_list)
df_domains['domain'] = df_domains.index
df_domains['valid'] = ''
df_domains['valid_domain'] = ''
df_domains['trust'] = ''

df_temp_domains = df_domains.copy()

df_domains

,domain,valid,valid_domain,trust
gmail.com,gmail.com,,,
outlook.com,outlook.com,,,
yahoo.com.ar,yahoo.com.ar,,,
pereirayasociados.com.ar,pereirayasociados.com.ar,,,
hotmail.com,hotmail.com,,,
...,...,...,...,...
ahmad.com.ar,ahmad.com.ar,,,
sinergylanguage.com,sinergylanguage.com,,,
synergylanguage.com,synergylanguage.com,,,
ddas.com.ar,ddas.com.ar,,,


In [96]:
# Setup Graphics df

dict_graph = {}

dict_temp_graph = dict_graph.copy()

In [97]:
dict_graph = dict_temp_graph.copy()

# Store graph data

dict_graph.update({'domains_total': domains.shape[0]})

dict_temp_graph = dict_graph.copy()
dict_graph

{'domains_total': 5309}

In [98]:
df_domains = df_temp_domains.copy()

# Verify existence for MX records in DNSs for each domain

domains = df_domains.index
domains_list = list(domains)

def validate_existence(domain):
    try:
        resolver.resolve(domain, 'MX')
    except:
        print('-', end='')
        df_domains.at[domain, 'valid'] = False
        df_domains.at[domain, 'valid_domain'] = domain
        df_domains.at[domain, 'trust'] = 0
    else:
        print('+', end='')
        df_domains.at[domain, 'valid'] = True
        df_domains.at[domain, 'valid_domain'] = domain
        df_domains.at[domain, 'trust'] = 100

with ThreadPoolExecutor() as executor:
    executor.map(validate_existence, domains_list)

df_temp_domains = df_domains.copy()

+++++++++++++++++++-++-+++++++++-+++++-++++++++++++++++++++++++++++++-++++++++++-+-+-++++++++++++++++++++++++++++++-++++++++-++++-+++-+++--++++++--+-++-++++++++++-+++++++++-+++++++++--+++++++++++++--++++++++-+++++++++++++++++-+++++++++++++++++-+++++++++++++++++++++++++++++++-+++++-++-+-++-++++++-++++++-+-+-+++++-++++++++++++++++++++++++++++++++++-++-++-+++-++++++-++-++++-++++++++++++++++++++-+-+++++-++++++++-+++++--+++++++++++++++++++++-++-++++-++-+++-+-++++++-++++++-++-++++++++++++++++-++-++++++++++-+++++++++---++++++++---++++++++-+++++++++++-+-++++++-++++++++++++++-+++++++-+++++++++-++-+--+++-+++++--+--++++-++++++++++++++++-+++-+++++++-+++++--++++-+-+++-++++++++++-+++-+++-+++++--++++++-+++-+++++++-++++-+++++++++++++++++--++++++++++++++++++++++++--+-+++++++++-+++-++++++++++-++-+-+++-++-+++++++++++-++++++++++-+++--++-++++++++-++++++++++++++++++++++++++-++-++++-+++++++++++++++++++++-+--+++++++-+++++-+++++++++-+++++++++--+++++++++--++-++++++++++++++-+-++++++++-++++++++++-+++++++++++-++++

In [99]:
dict_graph = dict_temp_graph.copy()

# Store graph data

dict_graph.update({'domains_MX_false': df_domains.valid.value_counts()[0]})
dict_graph.update({'domains_MX_true': df_domains.valid.value_counts()[1]})

dict_temp_graph = dict_graph.copy()
dict_graph

{'domains_total': 5309, 'domains_MX_false': 1146, 'domains_MX_true': 4163}

In [100]:
df_domains = df_temp_domains.copy()

# Verify existence of A records in DNSs for domains starting without 'www.'
# MX records fallback in A records.

domains = df_domains[df_domains.valid == False].index
domains_list = list(domains)

def validate_existence(domain):
    try:
        response = requests.get(f'http://{domain}', timeout=10)
    except:
        print('-', end='')
    else:
        print('+', end='')
        df_domains.at[domain, 'valid'] = True
        df_domains.at[domain, 'valid_domain'] = domain
        df_domains.at[domain, 'trust'] = 99

with ThreadPoolExecutor() as executor:
    executor.map(validate_existence, domains_list)

df_temp_domains = df_domains.copy()

---+-------+---+---+--+-----+--+---+------+-----+-------+--+---------+---------+---------------------------+-------+-------++--+----+------+------------+---++-----+---------------------------+-------++----------------+---+--------+-+--+------------+--------+---------------------------------------------------------+---++-+-+----+++------+---------+----+---+-------+-----------+-++------------------------------+-----+-----+-------------+----+-----------+------+--++----+------------++++--------------------+-----------++-----+--------+----+---++--+--+-------------+--+--------------------+---+----++------+---++-+--+--------+----------+---+----------+------+-------+++-+-----------+----+----+----------------+-------+---------+---------+-------+----+--------------------+--++-----------+--------+----------------------------------++-----+------------+--------------+--+-------+-----+----------+-+------------+----+----------+-----+-------+---------+-------------------+----+--------------++---------

In [101]:
df_domains = df_temp_domains.copy()

# Verify existence of A records in DNSs for domains starting with 'www.'
# MX records fallback in A records.

domains = df_domains[df_domains.valid == False].index
domains_list = list(domains)

def validate_existence(domain):
    try:
        response = requests.get(f'http://www.{domain}', timeout=10)
    except:
        print('-', end='')
    else:
        print('+', end='')
        df_domains.at[domain, 'valid'] = True
        df_domains.at[domain, 'valid_domain'] = domain
        df_domains.at[domain, 'trust'] = 98

with ThreadPoolExecutor() as executor:
    executor.map(validate_existence, domains_list)

df_domains.domain.value_counts()

df_temp_domains = df_domains.copy()

-------------------------------------------------+----------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [102]:
dict_graph = dict_temp_graph.copy()

# Store graph data

dict_graph.update({'domains_A_false': df_domains.valid.value_counts()[0]})
dict_graph.update({'domains_A_true': df_domains.valid.value_counts()[1]})

dict_temp_graph = dict_graph.copy()
dict_graph

{'domains_total': 5309,
 'domains_MX_false': 1146,
 'domains_MX_true': 4163,
 'domains_A_false': 992,
 'domains_A_true': 4317}

In [103]:
df_domains = df_temp_domains.copy()

# Replace by dictionary 1: 
#   Domains that are sinks for spam collection because of its short distance
#   with a TRUE valid domain and thus whould result in a FALSE POSITIVE 
#   high trust assignment.
#   https://tecnofimatica.com/hotmail/iniciar-sesion/#Dominios_regionales

by_hand_correct = [
                'hotmail.com',
                'hotmail.com.ar',
                'hotmail.es',
                'hotmail.cl',
                'hotmail.it',
                'hotmail.fr',
                'outlook.com',
                'outlook.es',
                'outlook.cl',
                'outlook.it',
                'outlook.fr',
                'live.com',
                'live.com.ar',
                'live.com.mx',
                'live.co',
                'live.fr',
                'gmail.com',
                'speedy.com.ar',
                'yahoo.com.ar',
                'yahoo.com.br',
                'yahoo.com.mx',
                'yahoo.com.es',
                'yahoo.com.tw',
                'yahoo.com.cn',
                'yahoo.com',
                'ymail.com',
                'yahoo.es',
                'yahoo.it',
                'yahoo.fr',
                'yahoo.ca'
                # add pairs as detected, see (*) Note.
]

by_hand_valid = [
                'mail.com',         # Suspicious of being false provider
                'email.com',        # Suspicious of being false provider
                # 'gotmail.com',      # Suspicious of being false provider
                'bahco.com',
                'life.com',
                'life.com.ar',
                'aivo.co',
                'gire.com',
                'riva.com.ar',
                'liv.com.ar',
                'tije.com',
                'tije.com.ar',
                'linde.com',
                'ilove.com',
                'line.com.ar',
                'lije.com'
                # add pairs as detected, see (*) Note.
]

domains = df_domains[df_domains.valid == True].index
domains_list = list(domains)

for domain in domains_list:
    for correct in by_hand_correct:
        if domain in by_hand_valid or domain in by_hand_correct:
            print('-', end='')
            continue                    # do not change, keep it.
        distance = lev(domain, correct)
        if distance != 0 and distance <= 2:
            if df_domains.at[domain, 'trust'] == 90:            # (*) TRUST 2°
                print('o', end='')      # a better distance has been assigned
                continue                # is already better, keep it
            print('+', end='')
            # (*) Note: enable the following print to detect domains that
            #           should not be changed...
            # print('\n', domain, '->', correct, '->', distance)
            df_domains.at[domain, 'valid_domain'] = correct
            df_domains.at[domain, 'valid'] = False
            df_domains.at[domain, 'trust'] = 91 - distance      # (*) TRUST 1°

df_temp_domains = df_domains.copy()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------+------------------------------+------------------------------------------------------------------------------------------------------------------------+------------------------------++o+o++o+o+o------------------------------+++oo+++++o++------------------------------+oo++------------------------------------------------------------------------------------------++o++o+o++++------------------------------+++o------------------------------+o++o------------------------------------------------------------+------------------------------------------------------------++++++o++------------------------------+o------------------------------+------------------------------+++++---------------------------

In [104]:
dict_graph = dict_temp_graph.copy()

# Store graph data

mask = (df_domains.trust <= 90) & (df_domains.trust >= 81)

dict_graph.update({'domains_spam_sink':  mask.value_counts()[1]})

dict_temp_graph = dict_graph.copy()
dict_graph

{'domains_total': 5309,
 'domains_MX_false': 1146,
 'domains_MX_true': 4163,
 'domains_A_false': 992,
 'domains_A_true': 4317,
 'domains_spam_sink': 91}

In [105]:
# Detect the '.com.com' trap

domains = df_domains.domain.unique()
domains_list = list(domains)

for domain in domains_list:
    if str(df_domains.loc[domain].domain).endswith(".com.com"):
        print(df_domains.loc[domain].domain)

hotmail.com.com
live.com.com


In [106]:
df_domains = df_temp_domains.copy()

# Replace by dictionary 2: 
# 1. known domains that have a higher distance than 2 and were not
#    replaced during the previous step (Replace by dictionary 1).
#    Levenshtein distance with correct addresses. 
# 2. domains that have changed because of company change.
# 3. get rid of the '.com.com' trap (spam sink)
# 4. domains that were incorrectly assigned because of its short distance 
#    with a TRUE valid domain (spam sink or valid domain) and thus whould 
#    result in a FALSE POSITIVE high trust assignment. 
# 5. domains that are TRUE, assumed to be wrong inputs, and should be corrected.

by_hand_dict = {
                'gmail.com.ar': 'gmail.com',                    # case 1
                'hotmail.com.es': 'hotmail.es',                 # case 1
                'hotmail.ar': 'hotmail.com.ar',                 # case 1
                'bacho.com': 'bahco.com',                       # case 1
                'tarjetashopping.com.ar': 'tarshop.com.ar',     # case 2
                'alpargatas.com.ar': 'topper.com',              # case 2
                'hotmail.com.com': 'hotmail.com',               # case 3
                'live.com.com': 'live.com',                     # case 3
                'gamial.com': 'gmail.com',                      # case 4
                'gmial.com': 'gmail.com',                       # case 4
                'bacs.com': 'bacs.com.ar'                       # case 5
                # add pairs as detected
} 

domains = df_domains.index
domains_list = list(domains)

for domain in domains_list:
    for key, value in by_hand_dict.items():
        if key == domain:
            df_domains.at[domain, 'valid_domain'] = value
            df_domains.at[domain, 'valid'] = False
            df_domains.at[domain, 'trust'] = 80
            print('+', end='')

df_temp_domains = df_domains.copy()

+++++++++++

In [107]:
dict_graph = dict_temp_graph.copy()

# Store graph data

mask = (df_domains.trust == 80)

dict_graph.update({'domains_known_sink':  mask.value_counts()[1]})

dict_temp_graph = dict_graph.copy()
dict_graph

{'domains_total': 5309,
 'domains_MX_false': 1146,
 'domains_MX_true': 4163,
 'domains_A_false': 992,
 'domains_A_true': 4317,
 'domains_spam_sink': 91,
 'domains_known_sink': 11}

In [108]:
df_domains = df_temp_domains.copy()

# Allocation of domains (with 0 TRUST) by [1, 2, 3] Levenshtein distance
# This step may produce FALSE POSITIVES, and < 40 trust should be 
# considered MEDIUM TRUST. 

domains_1 = df_domains[df_domains.valid == True].index
domains_list_1 = list(domains_1)
domains_2 = df_domains[df_domains.trust == 0].index
domains_list_2 = list(domains_2)

MAX_CHANGES = 3     # Trial and error proved this constant as best suited
for distance in range(1, MAX_CHANGES+1):
    for domain_2 in domains_list_2:
        if df_domains.at[domain_2, 'trust'] != 0:
            print('o', end='')      # a better distance has been assigned
            continue
        for domain_1 in domains_list_1:
            if domain_1 != domain_2 and lev(domain_1, domain_2) == distance:
                df_domains.at[domain_2, 'valid_domain'] = domain_1
                df_domains.at[domain_2, 'trust'] = 41-distance
                print('+-', end='')

df_temp_domains = df_domains.copy()

+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-ooo+-+-+-+-+-+-+-+-+-+-oooooo+-ooo+-+-+-+-+-+-+-+-+-+-ooooo+-+-+-+-+-+-+-+-+-+-+-+-+-ooo+-oo+-o+-+-+-+-+-o+-ooo+-o+-+-ooo+-+-+-+-ooo+-+-o+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-oo+-oo+-ooo+-o+-+-oo+-+-+-+-+-+-+-o+-+-+-ooo+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-o+-+-+-+-+-+-+-+-+-o+-+-+-+-+-ooo+-ooooooo+-+-+-+-+-+-+-+-+-+-+-+-+-o+-ooooo+-+-+-+-+-+

In [109]:
dict_graph = dict_temp_graph.copy()

# Store graph data

mask = (df_domains.trust <= 40) & (df_domains.trust >= 31)

dict_graph.update({'domains_medium_trust':  mask.value_counts()[1]})

dict_temp_graph = dict_graph.copy()
dict_graph

{'domains_total': 5309,
 'domains_MX_false': 1146,
 'domains_MX_true': 4163,
 'domains_A_false': 992,
 'domains_A_true': 4317,
 'domains_spam_sink': 91,
 'domains_known_sink': 11,
 'domains_medium_trust': 595}

In [110]:
df_domains = df_temp_domains.copy()

# Allocation of domains (with 0 TRUST) by MIN(LEN)/2 Levenshtein distance
# This step may produce FALSE POSITIVES, and < 30 trust should be 
# considered LOW TRUST. 
# ---
# For the purpose of the dataset under testing there were no interesting 
# outputs and the code was disabled. 

while(False):                           # Remove this line to enable the code.
    domains_1 = df_domains[df_domains.valid == True].index
    domains_list_1 = list(domains_1)
    domains_2 = df_domains[df_domains.trust == 0].index
    domains_list_2 = list(domains_2)

    contador = 0
    for domain_2 in domains_list_2:
        for domain_1 in domains_list_1:
            distance = lev(domain_1, domain_2)
            if df_domains.loc[domain_2].trust > 31-distance:     # (*) TRUST 2°
                continue    # a better distance has been assigned
            if distance < min(len(domain_1), len(domain_2)) / 2:
                df_domains.loc[domain_2, 'valid_domain'] = domain_1
                df_domains.loc[domain_2, 'trust'] = 31-distance  # (*) TRUST 1°
                contador += 1
                if (contador % 1000 == 0):
                    print(contador, end=' - ')
    print(contador)

df_temp_domains = df_domains.copy()

In [ ]:
df_domains = df_temp_domains.copy()
df_mails = df_temp_mails.copy()

# Merge Mails and Domains dfs

df_mails = pd.merge(df_mails, df_domains, how='right', on=['domain'])
df_mails.sort_values(by=['ID', 'ID_Mail'], inplace=True)

df_temp_mails = df_mails.copy()

# df_mails

In [112]:
# Save data for the next batch process

df_mails.to_csv('df_mails_batch1.csv')

with open("dict_graph1.dat", "wb") as outfile:
    pickle.dump(dict_graph, outfile)

In [113]:
str(datetime.now())

'2022-09-11 14:29:24.187843'